In [1]:
import os, pickle, random
import numpy as np
import torch

In [2]:
# change the current directory to the father directory of this file
os.chdir(os.path.dirname(os.path.dirname(os.path.realpath('evaluate.ipynb'))))
print(os.getcwd())

/home/yangshuaiyu6791/RepresentAttack/inversion_sePoint


In [3]:
from modules.models import sPoints_net, ePoints_net, se_net
from DataLoader.dataloader import TrajDataset
emb_dim = 128
emb_model = 'neutraj'

# load data
with open("./data/{}/{}/train/train_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    train_embs = tmp_embs['embs']
with open("./data/{}/{}/train/train_grid".format(emb_model, emb_dim), "rb") as f:
    train_grids = pickle.load(f)

with open("./data/{}/{}/val/val_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    val_embs = tmp_embs['embs']
with open("./data/{}/{}/val/val_grid".format(emb_model, emb_dim), "rb") as f:
    val_grids = pickle.load(f)
    
with open("./rdata/gps_seqs/traingps", "rb") as f:
    train_gps = pickle.load(f)
    train_gps = train_gps[0:60000]
with open("./rdata/gps_seqs/valgps", "rb") as f:
    val_gps = pickle.load(f)

In [4]:
# load pth
# delta = 0.002, emb_model = neutraj, embedding dim = emb_dim, hidden dim = 512

model = se_net(emb_dim, 512)
params = torch.load('./models/d0_002_train_sePoints_{}_{}_{}.pth'.format(emb_model, emb_dim, 512))
model.load_state_dict(params)

<All keys matched successfully>

In [9]:
# calculate the distance between two points with longitude and latitude
def distance(lat1, lon1, lat2, lon2):
    R = 6371.004    # km
    pi = 3.1415926
    dlat = (lat2 - lat1) * pi / 180
    dlon = (lon2 - lon1) * pi / 180
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(lat1 * pi / 180) * np.cos(lat2 * pi / 180) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = R * c
    return d

# transform the cell x,y to lon, lat
porto_lat_range = [40.953673,41.307945]
porto_lon_range = [-8.735152,-8.156309]
delta = 0.002

def grid2gps(grid):    # x -> lon, y -> lat
    grid_x = grid[0]; grid_y = grid[1]
    gps_x = porto_lon_range[0] + grid_x * delta
    gps_y = porto_lat_range[0] + grid_y * delta
    return [gps_x, gps_y]    # [lon, lat]

In [10]:
# choose 40 points by random
choice_list = np.random.choice(len(val_embs), 40, replace=False)
print([val_gps[i][0] for i in choice_list])    # lon, lat
choice_valgrids = [[val_grids[i][0][0], val_grids[i][0][1], val_grids[i][-1][0], val_grids[i][-1][1]] for i in choice_list]    # x, y
choice_valembs = [val_embs[i] for i in choice_list]
choice_valgps = [[val_gps[i][0], val_gps[i][-1]] for i in choice_list]    # [[lon, lat], [lon, lat]]
# get the prediction
model.cpu()
model.eval()
choice_predgrids = []
for i in range(len(choice_valembs)):    
    choice_valembs[i] = torch.Tensor(choice_valembs[i]).unsqueeze(0)
    choice_predgrids.append(model(choice_valembs[i]).cpu().detach().numpy()[0])

# for i in range(len(choice_predgrids)):
#     print(choice_predgrids[i], choice_valgrids[i], '\n')

# transform the prediction to lon, lat
choice_predgps = []
for i in range(len(choice_predgrids)):
    choice_predgps.append([grid2gps(choice_predgrids[i][0:2]), grid2gps(choice_predgrids[i][2:])])
print(choice_predgps)

[array([-8.60652 , 41.144616]), array([-8.612919, 41.153742]), array([-8.647722, 41.181597]), array([-8.673822, 41.149638]), array([-8.609796, 41.147946]), array([-8.605611, 41.15367 ]), array([-8.623008, 41.146641]), array([-8.622387, 41.175819]), array([-8.609679, 41.160411]), array([-8.674101, 41.153877]), array([-8.600283, 41.182704]), array([-8.604864, 41.161122]), array([-8.585604, 41.148504]), array([-8.628147, 41.157351]), array([-8.564229, 41.151501]), array([-8.606763, 41.14575 ]), array([-8.64405 , 41.158917]), array([-8.604288, 41.182641]), array([-8.606466, 41.147982]), array([-8.619912, 41.147973]), array([-8.663769, 41.151195]), array([-8.610984, 41.147397]), array([-8.627724, 41.175405]), array([-8.604783, 41.16114 ]), array([-8.639838, 41.15979 ]), array([-8.605917, 41.153616]), array([-8.60481 , 41.161131]), array([-8.610858, 41.145174]), array([-8.599725, 41.182695]), array([-8.630595, 41.154867]), array([-8.611335, 41.147676]), array([-8.656254, 41.164605]), array([

In [11]:
# calculate the distance between prediction and ground truth
start_dis = []; end_dis = []
for i in range(len(choice_predgps)):
    start_dis.append(distance(choice_predgps[i][0][1], choice_predgps[i][0][0], choice_valgps[i][0][1], choice_valgps[i][0][0]))
    end_dis.append(distance(choice_predgps[i][1][1], choice_predgps[i][1][0], choice_valgps[i][1][1], choice_valgps[i][1][0]))
print(start_dis)
print(end_dis)
print('average start distance: {} km'.format(np.mean(start_dis)))
print('average end distance: {} km'.format(np.mean(end_dis)))

with open("./results/{}_{}_{}_{}_dists".format(delta,emb_model, emb_dim, 512), "w") as f:
    f.write("choose 40 points by random\n")
    f.write("choice_list: {}\n".format(choice_list))
    f.write('average start distance: {} km'.format(np.mean(start_dis)))
    f.write('average end distance: {} km'.format(np.mean(end_dis)))
    f.write('\n')
    f.write('start distance: {}'.format(start_dis))
    f.write('\n')
    f.write('end distance: {}'.format(end_dis))

[0.6803851516098737, 0.7775844858034427, 1.1450435796856606, 0.3624714773959707, 0.7029552771542159, 0.860532888775138, 0.4776780287243325, 1.0936243790666926, 0.9597081584823192, 0.34236608598323204, 1.5365861691731189, 1.0174218810717448, 1.0788000888817242, 0.733256072850645, 1.5493665881343193, 0.6821694578078733, 0.7074593768256365, 1.2670508372778468, 0.7265045066373654, 0.5271850463719153, 0.49379558746651453, 0.7191853839481775, 1.1818818591692999, 1.0271328127550359, 0.7541292890038984, 0.8878671320738185, 1.0161632480417784, 0.6699615419870825, 1.2763845735035486, 0.6420575051520118, 0.657065387032398, 0.8060799320137471, 0.6678035338445815, 1.247590917658901, 0.8879967916218476, 0.7860269413229073, 1.2759945935663581, 0.8789461420071041, 0.6792852610093789, 0.7772252548181833]
[0.9966980520757861, 1.3800995226322563, 1.6230817163772462, 0.9885379901074188, 1.9721359277731307, 1.866528803597888, 1.0426487008729328, 1.3535000587222956, 1.2042084318602437, 1.1445804516069624, 2

In [12]:
# if you want, plot the prediction and ground truth on the map
import folium
my_map = folium.Map(location=[40.953673,-8.735152], zoom_start=12)
choice = np.random.choice(len(choice_predgps))
print(choice_predgps[choice], choice_valgps[choice])

folium.CircleMarker(location=[choice_predgps[choice][0][1], choice_predgps[choice][0][0]], radius=5, color='blue', fill=True).add_to(my_map)
folium.CircleMarker(location=[choice_predgps[choice][1][1], choice_predgps[choice][1][0]], radius=5, color='blue', fill=True).add_to(my_map)
folium.CircleMarker(location=[choice_valgps[choice][0][1], choice_valgps[choice][0][0]], radius=5, color='red', fill=True).add_to(my_map)
folium.CircleMarker(location=[choice_valgps[choice][1][1], choice_valgps[choice][1][0]], radius=5, color='red', fill=True).add_to(my_map)

# plot the real trajectory
gps_val = val_gps[choice_list[choice]][:, [1, 0]]
folium.PolyLine(gps_val, color="red", weight=2.5, opacity=1).add_to(my_map)

my_map   # show the map

[[-8.612007094909668, 41.15371950878907], [-8.632512397338866, 41.15739789929199]] [array([-8.604864, 41.161122]), array([-8.628741, 41.169798])]


: 